# Notebook 2: Table Creation
Create Unity Catalog-managed Delta tables using corrected SQL syntax.

%md
# 📘 Delta Table Creation: Unity Catalog Setup

This notebook sets up all required Delta tables for the Aircraft AOG Demo inside Unity Catalog.

## ✅ Tables Created

1. **`raw_sensor_data`**  
   Stores synthetic sensor readings from aircraft systems.

2. **`maintenance_events`**  
   Stores aircraft maintenance events (e.g., routine checks, engine repairs).

3. **`cleaned_sensor_data`**  
   Created by DLT pipeline. Contains filtered, validated sensor data.

4. **`prediction_results`**  
   Created by model pipeline. Stores risk scores for each aircraft.

> All tables are created in:  
> `CATALOG: arao`, `SCHEMA: aerodemo`

In [0]:
%sql

-- Set correct catalog and schema
USE CATALOG arao;
USE SCHEMA aerodemo;

-- Create raw sensor data table
CREATE TABLE IF NOT EXISTS raw_sensor_data (
  timestamp TIMESTAMP,
  aircraft_id STRING,
  model STRING,
  engine_temp DOUBLE,
  fuel_efficiency DOUBLE,
  vibration DOUBLE
)
USING DELTA
COMMENT 'Raw sensor readings from synthetic generator';

-- Create maintenance events table
CREATE TABLE IF NOT EXISTS maintenance_events (
  aircraft_id STRING,
  event_date DATE,
  event_type STRING
)
USING DELTA
COMMENT 'Maintenance logs per aircraft';

-- Cleaned sensor data table (DLT will populate)
CREATE TABLE IF NOT EXISTS cleaned_sensor_data (
  timestamp TIMESTAMP,
  aircraft_id STRING,
  model STRING,
  engine_temp DOUBLE,
  fuel_efficiency DOUBLE,
  vibration DOUBLE
)
USING DELTA
COMMENT 'Cleaned sensor data after applying quality checks';

-- Prediction output table (DLT will populate)
CREATE TABLE IF NOT EXISTS prediction_results (
  aircraft_id STRING,
  prediction_date DATE,
  risk_score DOUBLE
)
USING DELTA
COMMENT 'Predicted AOG risk scores from model';
